In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 15 15:06:25 2023

@author: shaurya
"""

 

#Import all required libraries
import spacy
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import pandas as pd



import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

def dataListCreator(filePath) : 
    with open(filePath)  as f :
        data = f.readlines()
        dataList =[]

        for line in data :
            linelist=[]
            linelist = line.strip('\n').split('\t') 
            dataList.append(linelist) 
    

    return dataList

 
def sentenceProcessor(dataList) :
   
    count = 0 
    sentence=''
    #[(start_of_word1, end_of_word1, LABEL_for_word1), (start_of_word2, end_of_word2, LABEL_for_word2)….]
    entityList =[]
    for li in dataList :
        if len(li)>1 :
            if li[1]=='O' :
                sentence = sentence+li[0]+' '
                count = count+len(li[0])+1
            if li[1] != 'O':
                start = count
                sentence = sentence+li[0]+' '
                count = count+len(li[0])+1
                end = count-1
                label= li[1]+"_Disease"
                tup = (start,end,label)
                entityList.append(tup)
    return ([sentence ,{'entities':entityList}] )      
#sentenceProcessor(x)

def senetnceIsolator(dataFrame) :
    startIndex = 0
    endIndex = 0
    
    lisofFullStops = dataFrame.loc[dataFrame['words'] =='.'].index.tolist()
    start_End_List = []
    for i in range(len(lisofFullStops)) :
        if i == 0 :
            startIndex = 0
            endIndex = lisofFullStops[i]+1
            start_End_List.append((startIndex,endIndex))
        else :
            startIndex = endIndex+1
            endIndex = lisofFullStops[i]+1
            start_End_List.append((startIndex,endIndex))
            
    return  start_End_List

 


def prepareTrainingData(filePath) :
    
    dataList =  dataListCreator(filePath) # list containing list of sentence fragments 
    df = pd.DataFrame(dataList, columns = ['words', 'tags'])
    startEndLIst =senetnceIsolator(df)    # list of tuples , each containing start and end of the sentence index.
    dataset=[]
    for i in range(len(startEndLIst)) :
        start = startEndLIst[i][0]
        end = startEndLIst[i][1]
        #print(start,end)
        dataset.append(sentenceProcessor(dataList[start:end]))
    return dataset 

testdataset = prepareTrainingData( "E:/DATA/UNZIPPED/ner_tsv_Disease/test.tsv")
traindataset = prepareTrainingData( "E:/DATA/UNZIPPED/ner_tsv_Disease/train.tsv")
traindev = prepareTrainingData( "E:/DATA/UNZIPPED/ner_tsv_Disease/train_dev.tsv")

